## Imports and constants 

In [1]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

import pandas as pd
import numpy as np
from scipy import stats
import pickle
import os
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from tqdm import tqdm_notebook as tqdm
import glob
import gc

In [2]:
data_dir = '../data/'
outdir = data_dir + 'non_dl_processed/'
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
second_earthquake = 50085877
test_len = 150000

## Prepare datasets

The second earthquake happens at 50085877, so we can use the data before that (~8%) as validation data to evaluate different models with.

In [3]:
train = pd.read_csv(data_dir + 'train.csv')
train.shape

(629145480, 2)

In [4]:
valid = train[:second_earthquake + 1]
train = train[second_earthquake + 1:]

In [5]:
valid.to_csv(outdir + 'valid.csv', index=None)

In [6]:
train.to_csv(outdir + 'train.csv', index=None)

## Feature engineering

### Functions

In [6]:
def make_chunk_features(seq, chunk_size=1000):

    n_chunks = len(seq) // chunk_size
    features = []

    for i in range(n_chunks):

        chunk = seq[i * chunk_size : (i+1) * chunk_size]

        chunk_features = process_chunk(chunk)
        chunk_features.extend(process_chunk(chunk[-100:]))
        chunk_features.extend(process_chunk(chunk[-10:]))
        
        features.append(chunk_features)

    return np.array(features)


def process_chunk(chunk):

        mean = chunk.mean()
        std = chunk.std()
        var = np.square(std)
        q1 = np.quantile(chunk, 0.25)
        q3 = np.quantile(chunk, 0.75)
        arg_sorted = chunk.argsort()
        top3 = arg_sorted[-3:]
        bottom3 = arg_sorted[:3]
        skew = stats.skew(chunk)
        kurt = stats.kurtosis(chunk)

        minimum = chunk.min()
        maximum = chunk.max()

        features = [
            mean,
            std,
            q1,
            q3,
            var,
            top3[0],
            top3[1],
            top3[2],
            bottom3[0],
            bottom3[1],
            bottom3[2],
            skew,
            kurt,
            minimum,
            maximum
        ]

        return features

### Train data

In [5]:
train = pd.read_csv(outdir + 'train.csv')
valid = pd.read_csv(outdir + 'valid.csv')
train.shape, valid.shape

((579059603, 2), (50085877, 2))

In [20]:
def engineer_features(df, step=test_len):
    
    features = []
    labels = []
    for i in tqdm(range((len(df) // step) - (test_len // step) + 1)):

        cur_slice = df[i * step : i * step + test_len]
        time_to_failure = cur_slice['time_to_failure'].values
        if time_to_failure[-1] > time_to_failure[0]:
            continue

        features.append(make_chunk_features(cur_slice['acoustic_data'].values))
        labels.append(time_to_failure[-1])

    features = np.array(features)
    labels = np.array(labels)
    
    return features, labels

In [21]:
gc.enable()
for n in [1, 3, 15]:
    for name, data in zip(['train', 'valid'], [train, valid]):
        step = test_len // n
        print("Engineering features for {} with step {:,}".format(name, step))
        features, labels = engineer_features(data, step=step)
        
        with open(outdir + '{}_features_{}.pkl'.format(name, n), 'wb') as f:
            pickle.dump(features, f)

        with open(outdir + '{}_labels_{}.pkl'.format(name, n), 'wb') as f:
            pickle.dump(labels, f)
        
        del features, labels
        gc.collect()

Engineering features for train with step 150,000



Engineering features for valid with step 150,000



Engineering features for train with step 50,000



Engineering features for valid with step 50,000



Engineering features for train with step 10,000



Engineering features for valid with step 10,000


### Submission data

In [6]:
test_features = []
seg_ids = []
test_files = glob.glob(data_dir + 'test/*')
for i, f in tqdm(enumerate(test_files)):
    df = pd.read_csv(f)
    test_features.append(engineer_features(df['acoustic_data'].values))
    seg_ids.append(f.split('/')[-1].replace('.csv', ''))
    
test_features = np.array(test_features)
test_features.shape

(2624, 150, 45)

In [7]:
with open(outdir + 'test.pkl', 'wb') as f:
    pickle.dump(test_features, f)
    
with open(outdir + 'test_ids.pkl', 'wb') as f:
    pickle.dump(seg_ids, f)

## Machine learning

In [120]:
def load_raw_data(version='1'):

    with open(outdir + 'train_features_{}.pkl'.format(version), 'rb') as f:
        features_raw = pickle.load(f)

    with open(outdir + 'train_labels_{}.pkl'.format(version), 'rb') as f:
        labels_raw = pickle.load(f)

    with open(outdir + 'valid_features_1.pkl', 'rb') as f:
        test_features_raw = pickle.load(f)

    with open(outdir + 'valid_labels_1.pkl', 'rb') as f:
        test_labels_raw = pickle.load(f)
        
    return features_raw, labels_raw, test_features_raw, test_labels_raw

### LGBM

#### Loading data

In [ ]:
features_raw, labels_raw, test_features_raw, test_labels_raw = load_raw_data(version='3')

In [37]:
def preprocessing_lgbm(features_raw):
    
    features_1 = features_raw[:, 0, :]
    features_2 = features_raw[:, -1, :]
    features_3 = features_raw.mean(axis=1)
    features = np.concatenate([features_2, features_3], axis=1)    
    
    return features

In [38]:
features = preprocessing_lgbm(features_raw)
test_features = preprocessing_lgbm(test_features_raw)

#### Training

In [40]:
def assert_numpy(arr):

    if isinstance(arr, np.ndarray):
        return arr
    elif isinstance(arr, list):
        return np.array(arr)
    elif isinstance(arr, pd.Series):
        return arr.values
    else:
        print(arr)
        raise TypeError('Non array format, can\'t convert to numpy.')


def compute_l1(y_true, preds):
    ''' Computes the mean absolute error, or l1 score
    Range: [0, inf]
    '''
    y_true = assert_numpy(y_true)
    preds = assert_numpy(preds)

    return np.abs(y_true - preds).mean()


def compute_l2(y_true, preds):
    ''' Computes the mean squared error
    Range: [0, inf]
    '''
    y_true = assert_numpy(y_true)
    preds = assert_numpy(preds)

    return np.square(y_true - preds).mean()


def get_metrics():
    
    metrics = {
        'l1' : compute_l1,
        'l2' : compute_l2,
    }
    
    return metrics


def ensemble_predict(models, X_test):

    preds = np.zeros((len(models), len(X_test)))
    for i, m in enumerate(models):
        preds[i] = m.predict(X_test)

    return preds.mean(axis=0), preds.std(axis=0)


def evaluate_model(model, X_valid, Y_valid):
    
    ## Get predictions
    preds = model.predict(X_valid, num_iteration=model.best_iteration_)
    
    ## Compute metrics such as auc, frequency etc.
    metrics = get_metrics()
    evals = {}
    
    for m in metrics.keys():
        evals[m] = metrics[m](Y_valid, preds)
    
    return evals


def evaluate_ensemble(models, X_test, Y_test):
    
    metrics = get_metrics()
    eval_lists = {m : [] for m in metrics.keys()}
    for model in models:
        evals = evaluate_model(model, X_test, Y_test)
        for m in evals.keys():
            eval_lists[m].append(evals[m])
            
    mean_evals = {m : np.mean(eval_lists[m]) for m in eval_lists.keys()}
    
    return mean_evals
    

def train_ensemble(lgbparams, features, targets, nfold=10, test_size=0.1):
    
    kfold = KFold(n_splits=nfold, shuffle=False)
    metrics = get_metrics()
    eval_lists = {m : [] for m in metrics.keys()}
    feature_importances_df = pd.DataFrame()
    models = []

    for i, (train_index, test_index) in enumerate(kfold.split(features, targets)):
        
        ## Extract training and validation data
        X_train, X_valid = features[train_index], features[test_index]
        Y_train, Y_valid = targets[train_index], targets[test_index]

        ## Train lgb model
        model = lgb.LGBMRegressor(**lgbparams)
        model.fit(X_train, Y_train, eval_metric=lgbparams['metric'],
                      eval_set=[(X_valid, Y_valid), (X_train, Y_train)],
                      eval_names=['valid', 'train'],
                      early_stopping_rounds=100, verbose=False)
        
        ## Evaluate model
        evals = evaluate_model(model, X_valid, Y_valid)
        
        for m in evals.keys():
            eval_lists[m].append(evals[m])
            
        models.append(model)
    
    mean_evals = {m : np.mean(eval_lists[m]) for m in eval_lists.keys()}
    
    return models, mean_evals

In [41]:
lgbparams = dict(
    random_state = 1,
    objective = 'regression_l1',
    metric = 'mae',
    learning_rate = 0.3,
    num_leaves = 17,
    max_depth = 9,
    num_boosting_rounds = 10000,
)

In [42]:
models, evals = train_ensemble(lgbparams, features, labels_raw)

In [43]:
evaluate_ensemble(models, test_features, test_labels_raw)

{'l1': 2.0157322850187027, 'l2': 6.03985328628869}

#### Testing

In [74]:
with open(outdir + 'test.pkl'.format(version), 'rb') as f:
    test_features = pickle.load(f)
    
with open(outdir + 'test_ids.pkl'.format(version), 'rb') as f:
    ids = pickle.load(f)

In [75]:
test_features = test_features.mean(axis=1)

In [77]:
preds, _ = ensemble_predict(models, test_features)
preds = pd.DataFrame({'seg_id' : ids, 'time_to_failure' : preds})

In [80]:
preds.to_csv('../submission.csv', index=None)

### LSTM

In [93]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#### Preprocess data for LSTM

In [121]:
features_raw, labels_raw, test_features_raw, test_labels_raw = load_raw_data(version='3')

In [122]:
features = features_raw.copy()
labels = labels_raw.copy()

test_features = test_features_raw.copy()
test_labels = test_labels_raw.copy()

In [123]:
def truncate_features(features, chunk_size=10):
    new_features = []
    chunk_size = 10
    for f in features:
        new_f = []
        for s in range(features.shape[1] // chunk_size):
            f_s = f[s*chunk_size : (s+1)*chunk_size].mean(axis=0)
            new_f.append(f_s)
        new_features.append(new_f)

    new_features = np.array(new_features)
    
    return new_features

In [124]:
## Make smaller
features = truncate_features(features)
test_features = truncate_features(test_features)

In [125]:
def scale_features(features, scaler=None):
    
    prev_shape = features.shape
    features = features.reshape(-1, features.shape[-1])
    if scaler is None:
        scaler = MinMaxScaler()
        scaler.fit(features)
    features = scaler.transform(features)
    features = features.reshape(prev_shape)
    
    return features, scaler

In [79]:
## Scale
features, scaler = scale_features(features)
test_features, scaler = scale_features(test_features, scaler)

#### Load data

In [126]:
def get_loader(feats, labels, shuffle=False, batch_size=32):

    features = torch.tensor(feats, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    dataset = TensorDataset(features, labels)
    data_loader = DataLoader(dataset, shuffle=shuffle, batch_size=batch_size)
    
    return data_loader

In [127]:
X_train, X_valid, Y_train, Y_valid = train_test_split(features, labels, test_size=0.1, shuffle=False)
train_loader = get_loader(X_train, Y_train, shuffle=True, batch_size=32)
valid_loader = get_loader(X_valid, Y_valid, shuffle=False, batch_size=128)
test_loader = get_loader(test_features, test_labels, shuffle=False, batch_size=128)

In [138]:
n_data, seq_len, n_features = X_train.shape
X_train.shape

(10382, 15, 45)

##### Custom loader

In [194]:
import importlib 
import utils.dataset
importlib.reload(utils.dataset)
from utils.dataset import *

train_data = train.values
del train

In [287]:
seq_len, n_features = engineer_features(train_data[0:test_len, 0]).shape

X_train = train_data[second_earthquake + 1:]
X_valid = train_data[:second_earthquake + 1]

train_step = 50000
batch_size = 32
train_dataset = EarthquakeDatasetTrain(X_train, window_step=train_step, mask_prob=0.0)
valid_dataset = EarthquakeDatasetTrain(X_valid, window_step=150000)

train_loader = RandomLoader(train_dataset, 
                            batch_size=batch_size,
                            num_epoch_steps=int(len(X_train) / train_step / batch_size))

valid_loader = DataLoader(valid_dataset, 
                          batch_size=64, 
                          shuffle=False)

#### Create model

In [129]:
class LSTMNet(nn.Module):
    
    def __init__(self, config):
        super(LSTMNet, self).__init__()
        
        self.config = config
        self.device = torch.device("cuda" if self.config['use_cuda'] else "cpu")
        self.rnn = nn.GRU(config['n_features'], config['hidden_size'],
            batch_first=True,
            num_layers=config['num_layers'], 
            bidirectional=config['bidirectional'],
            dropout=config['dropout'])
        lstm_outsize = config['hidden_size'] * (1+1*config['bidirectional'])
        self.dropout = nn.Dropout(config['dropout'])
        self.dense = nn.Linear(lstm_outsize, config['dense_size'])
        self.classifier = nn.Linear(config['dense_size'], 1)
        self.criterion = nn.L1Loss()
        
    def forward(self, x, labels=None):
        x, hidden = self.rnn(x)
        x = self.dropout(x[:, -1, :])
        x = self.dropout(nn.functional.relu(self.dense(x)))
        x = self.classifier(x)

        if labels is not None:
            return x, self.criterion(x, labels)
        else:
            return x

In [131]:
config = dict(
    use_cuda = torch.cuda.is_available(),
    seq_len = seq_len,
    n_features = n_features,
    
    lr = 0.005,
    num_layers = 1,
    bidirectional = False,
    hidden_size = 256,
    dropout = 0.3,
    dense_size = 256,
)
device = torch.device('cuda' if config['use_cuda'] else 'cpu')

In [132]:
model = LSTMNet(config).to(device)
print('{:,} parameters'.format(sum([np.prod(param.shape) for param in model.parameters()])))

298,753 parameters


/home/aiwizo/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [133]:
params = model.parameters()
optimizer = torch.optim.Adam(params, lr=config['lr'])

#### Train

In [134]:
def eval_loader(model, loader):
    
    model.eval()
    tot_loss = tot_examples = 0
    with torch.no_grad():
        for batch in loader:
            
            if isinstance(batch, dict):
                features = batch['features'].to(device)
                labels = batch['labels'].to(device)
            else:
                features = batch[0].to(device)
                labels = batch[1].to(device)
                
            batch_size = len(features)

            output, loss = model(features, labels)

            tot_loss += loss.cpu().item() * batch_size
            tot_examples += batch_size
        
    model.train()
    
    return tot_loss / tot_examples

In [135]:
def train(model, train_loader, valid_loader, num_epochs=20, progress_bar=False):
    model.train()
    print("loss before training:")
    print(eval_loader(model, valid_loader))
    print()
    for e in range(num_epochs):
        if progress_bar:
            pbar = tqdm(total=len(train_loader))
        for batch in train_loader:
            
            if isinstance(batch, dict):
                features = batch['features'].to(device)
                labels = batch['labels'].to(device)
            else:
                features = batch[0].to(device)
                labels = batch[1].to(device)

            output, loss = model(features, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if progress_bar:
                pbar.update(1)
        
        print("loss after epoch {}:".format(e+1))
        print(eval_loader(model, valid_loader))
        print()

In [136]:
train(model, train_loader, valid_loader, num_epochs=20)

loss before training:
4.818936448783147

loss after epoch 1:
3.2542179064692824

loss after epoch 2:
2.980502081495844

loss after epoch 3:
2.8848164193138506

loss after epoch 4:
3.0910143009298183

loss after epoch 5:
3.1977603530553225

loss after epoch 6:
2.9255593356061436

loss after epoch 7:
3.0660766596604057

loss after epoch 8:
3.0744742743890456

loss after epoch 9:
3.2512747319980138

loss after epoch 10:
3.4239871134171245

loss after epoch 11:
3.085137952345083

loss after epoch 12:
3.1231004369527464

loss after epoch 13:
3.3154710045720304

loss after epoch 14:
3.113197033285475

loss after epoch 15:
3.1412166210353063

loss after epoch 16:
2.884876691695929

loss after epoch 17:
3.2580325830747183

loss after epoch 18:
3.212992804509102

loss after epoch 19:
3.1762593127206045

loss after epoch 20:
3.27195335715441



In [137]:
eval_loader(model, test_loader)

3.1881400935621147